****
**Goal:**
Practice real-world data cleaning and validation using Python and pandas—focusing on the steps an analyst or data engineer would take before loading data into a pipeline.

**Dataset:** Student Performance Data

(395 rows × 33 columns, secondary school student data)

**Key Tasks:**

Load CSV file from /kaggle/input/

Inspect column structure, data types, and null values

Remove duplicates and standardize column names

Validate numeric ranges (age, grades, alcohol use)

Create automated checks and an “at-risk” flag

Save cleaned and augmented datasets to /kaggle/working/

**Outputs Generated:**

File	Purpose
student_performance_cleaned.csv	Main cleaned dataset
student_performance_cleaned_augmented.csv	Cleaned + engineered columns
summary_numeric.csv	Numeric profile (stats summary)
frequencies.csv	Category frequencies
flagged_at_risk_students.csv	Students flagged by ETL rules
report.txt	Quick data-quality summary

**Learning Notes:**

Practicing df.info(), df.describe(), and isna().sum() builds data intuition.

Even clean datasets can benefit from standardization and basic validation.

Writing outputs to /working/ mimics staging/ETL data handoffs.

Adding automation (like flags and reports) brings structure to analytics workflows.

In [5]:
# Import essential libraries 
import pandas as pd

# Load the dataset
file_path = '/kaggle/input/student-performance-data/student_data.csv'
df = pd.read_csv(file_path)

#Inspect the first few rows
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [5]:
# Inspect data structure and quality 

# Basic information about columns, types and non-null counts
df.info()

# Quick statistical summary (only for numeric columns)
df.describe()

# Count of missing values per column 
df.isna().sum()

# Check for duplicate rows
df.duplicated().sum()

# Display column names for reference
df.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']

In [6]:
# Basic cleaning - duplicates and formatting

# Check and remove duplicates
duplicates = df.duplicated().sum()
print(f"Duplicate rows found: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found.")

# Standardize column name (optional but good practice)
df.columns = df.columns.str.strip().str.lower()

# Verify column name changes
print("\nUpdated column names:\n", df.columns.tolist())

# Confirm shape after cleaning 
print(f"\nDataset shape after cleaning: {df.shape}")

Duplicate rows found: 0
No duplicates found.

Updated column names:
 ['school', 'sex', 'age', 'address', 'famsize', 'pstatus', 'medu', 'fedu', 'mjob', 'fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'dalc', 'walc', 'health', 'absences', 'g1', 'g2', 'g3']

Dataset shape after cleaning: (395, 33)


In [7]:
# Data validation and sanity checks

# Age should be between 10 and 22
invalid_ages = df[(df['age'] < 10) | (df['age'] > 22)]
print("Unusual ages:\n", invalid_ages['age'].unique())

# Grades (g1, g2, g3) should be between 0 and 20

for col in ['g1', 'g2', 'g3']:
    invalid = df[(df[col] < 0) | (df[col] > 20)]
    if not invalid.empty:
        print(f"Unexpected values in {col}: {invalid[col].unique()}")

# Alcohol use score (dalc, walc) should be between 1 and 5
for col in ['dalc', 'walc']:
    out_of_range = df[(df[col] < 1) | (df[col] > 5)]
    if not out_of_range.empty:
        print(f"Out-of-range values in {col}: {out_of_range[col].unique()}")

# Quick look at category value counts for sanity 
print("\nUnique values in 'sex':", df['sex'].unique())
print("\nUnique values in 'school':", df['school'].unique())
print("\nUnique values in 'address':", df['address'].unique())

Unusual ages:
 []

Unique values in 'sex': ['F' 'M']

Unique values in 'school': ['GP' 'MS']

Unique values in 'address': ['U' 'R']


In [8]:
# Save the cleaned dataset

output_path = '/kaggle/working/student_performance_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")

Cleaned dataset saved to: /kaggle/working/student_performance_cleaned.csv


In [9]:
# Automation — quick summary + "at_risk" flag

import numpy as np
import pandas as pd # safe if already imported 

# Numeric summary (describe each numeric column)
numeric_cols = df.select_dtypes(include='number').columns
summary = df[numeric_cols].describe().T
summary_path = '/kaggle/working/summary_numeric.csv'
summary.to_csv(summary_path)

# Category frequency tables for quick data checks
cat_cols = ['sex', 'school', 'address', 'famsize', 'pstatus', 'mjob', 'fjob',
            'schoolsup', 'famsup', 'internet']
freqs = {}
for c in cat_cols:
    if c in df.columns:
        freqs[c] = df[c].value_counts(dropna=False)

# Save category frequencies to a single wide CSV
freq_df = pd.concat({k: v for k, v in freqs.items()}, axis=1)
freq_path = '/kaggle/working/frequencies.csv'
freq_df.to_csv(freq_path)

# Simple rule-based "at-risk" flag (ETL-style feature engineering)
#    - low final grade (g3 < 10)
#    - steady decline across terms (g1 > g2 > g3) with a drop of 4+ points total
#    - high absences (>= 10)
#    - prior failures (> 0)
#    - very high weekend alcohol use (walc >= 4)
df['grade_mean'] = df[['g1', 'g2', 'g3']].mean(axis=1)
df['grade_decline'] = (df['g1'] > df['g2']) & (df['g2'] > df['g3'])
df['high_absences'] = df['absences'] >= 10
df['prior_failures'] = df['failures'] > 0
df['high_weekend_alc'] = df['walc'] >= 4

df['at_risk'] = (
    (df['g3'] < 10) |
    (df['grade_decline'] & (df['g1'] - df['g3'] >= 4)) |
    df['high_absences'] |
    df['prior_failures'] |
    df['high_weekend_alc']
)

# Save flagged subset and augmented cleaned dataset
flagged = df[df['at_risk']].copy()
flagged_path = '/kaggle/working/flagged_at_risk_students.csv'
flagged.to_csv(flagged_path, index=False)

augmented_path = '/kaggle/working/student_performance_cleaned_augmented.csv'
df.to_csv(augmented_path, index=False)

# Tiny human-readable report (quick win for stakeholders)
report_lines = [
    f"Total rows: {len(df)}",
    f"At-risk students: {flagged.shape[0]}",
    f"Avg G3: {df['g3'].mean():.2f}",
    f"Median absences: {df['absences'].median()}",
    f"Share with prior failures: {df['prior_failures'].mean():.1%}",
]
report_path = '/kaggle/working/report.txt'
with open(report_path, 'w') as f:
    f.write('\n'.join(report_lines))

print("Saved:")
print("-", summary_path)
print("-", freq_path)
print("-", flagged_path)
print("-", augmented_path)
print("-", report_path)


Saved:
- /kaggle/working/summary_numeric.csv
- /kaggle/working/frequencies.csv
- /kaggle/working/flagged_at_risk_students.csv
- /kaggle/working/student_performance_cleaned_augmented.csv
- /kaggle/working/report.txt


**Reflection:**
This project helped me slow down and really understand what’s happening behind each pandas command instead of just copying code. I got to practice how data engineers clean and validate datasets before they’re used for analysis. It was simple but rewarding to see the process from inspection all the way to saving clean, structured data and building a small automation that feels practical for real work.